# 02_Preprocesamiento — Pipeline reproducible

# Importación de librerías

In [1]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline as SKPipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    make_scorer,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)

# Lectura del dataset

In [3]:
df = pd.read_csv('../data/diabetes_dataset.csv')
df.head()

,age,gender,ethnicity,education_level,income_level,employment_status,smoking_status,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,...,hdl_cholesterol,ldl_cholesterol,triglycerides,glucose_fasting,glucose_postprandial,insulin_level,hba1c,diabetes_risk_score,diabetes_stage,diagnosed_diabetes
0,58,Male,Asian,Highschool,Lower-Middle,Employed,Never,0,215,5.7,...,41,160,145,136,236,6.36,8.18,29.6,Type 2,1
1,48,Female,White,Highschool,Middle,Employed,Former,1,143,6.7,...,55,50,30,93,150,2.00,5.63,23.0,No Diabetes,0
2,60,Male,Hispanic,Highschool,Middle,Unemployed,Never,1,57,6.4,...,66,99,36,118,195,5.07,7.51,44.7,Type 2,1
3,74,Female,Black,Highschool,Low,Retired,Never,0,49,3.4,...,50,79,140,139,253,5.28,9.03,38.2,Type 2,1
4,46,Male,White,Graduate,Middle,Retired,Never,1,109,7.2,...,52,125,160,137,184,12.74,7.20,23.5,Type 2,1


# Depuración de variables y definición de X y Y

In [4]:
cols_to_drop = ['diabetes_stage', 'cholesterol_total', 'glucose_postprandial']
df = df.drop(columns=cols_to_drop, errors='ignore')

target_col = 'diagnosed_diabetes'

if target_col not in df.columns:
    raise ValueError(f"La columna objetivo '{target_col}' no existe. Revisa el nombre en df.columns.")

X = df.drop(columns=[target_col])
y = df[target_col].copy()

print("Shape completo X:", X.shape)
print("Distribución de la variable respuesta:")
print(y.value_counts(normalize=True))

Shape completo X: (100000, 27)
Distribución de la variable respuesta:
diagnosed_diabetes
1    0.59998
0    0.40002
Name: proportion, dtype: float64


# División en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)
print("\nProporción en train:")
print(y_train.value_counts(normalize=True))
print("\nProporción en test:")
print(y_test.value_counts(normalize=True))

Train: (80000, 27) Test: (20000, 27)

Proporción en train:
diagnosed_diabetes
1    0.599975
0    0.400025
Name: proportion, dtype: float64

Proporción en test:
diagnosed_diabetes
1    0.6
0    0.4
Name: proportion, dtype: float64


# Identificación de tipos de variables

In [6]:
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

binary_cols = ['family_history_diabetes', 'hypertension_history', 'cardiovascular_history']

numeric_to_scale = [col for col in num_cols if col not in binary_cols]

print("Numéricas a escalar:", numeric_to_scale)
print("Binarias (sin escalar):", binary_cols)
print("Categóricas:", cat_cols)

Numéricas a escalar: ['age', 'alcohol_consumption_per_week', 'physical_activity_minutes_per_week', 'diet_score', 'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi', 'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate', 'hdl_cholesterol', 'ldl_cholesterol', 'triglycerides', 'glucose_fasting', 'insulin_level', 'hba1c', 'diabetes_risk_score']
Binarias (sin escalar): ['family_history_diabetes', 'hypertension_history', 'cardiovascular_history']
Categóricas: ['gender', 'ethnicity', 'education_level', 'income_level', 'employment_status', 'smoking_status']


# Transformaciones: escalado y one-hot encoding

In [7]:
numeric_transformer = SKPipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = SKPipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num_scaled', numeric_transformer, numeric_to_scale),
    ('num_binary', 'passthrough', binary_cols),
    ('cat', categorical_transformer, cat_cols)
])

# Pipeline de prueba: regresión logística con class_weight='balanced'

In [8]:
os.makedirs('../models', exist_ok=True)

pipe_logreg_bal = SKPipeline(steps=[
    ('preproc', preprocessor),
    ('clf', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        random_state=42
    ))
])

pipe_logreg_bal.fit(X_train, y_train)

joblib.dump(pipe_logreg_bal, os.path.join('..', 'models', 'pipeline_logreg_classweight.joblib'))
print("Pipeline base (Regresión Logística con class_weight='balanced') entrenado y guardado.")

Pipeline base (Regresión Logística con class_weight='balanced') entrenado y guardado.


# GridSearchCV para optimizar la logística (preprocesamiento + modelo)

In [9]:
pipe_logreg_bal = SKPipeline(steps=[
    ('preproc', preprocessor),
    ('clf', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        random_state=42
    ))
])

**Nota:** se seleccionó el pipeline con class_weight='balanced' porque el dataset no presenta un desbalance de clases significativo, y esta opción permite compensar las leves diferencias en las proporciones sin modificar la estructura de los datos, manteniendo la reproducibilidad, comparabilidad y robustez del modelo.

In [10]:
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l2'],
    'clf__solver': ['lbfgs', 'liblinear']
    }

In [ ]:
scorer = make_scorer(recall_score)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [12]:
gs = GridSearchCV(
    estimator=pipe_logreg_bal,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv,
    n_jobs=-1,
    verbose=1
)

gs.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'clf__C': [0.01, 0.1, ...], 'clf__penalty': ['l2'], 'clf__solver': ['lbfgs', 'liblinear']}"
,scoring,make_scorer(r...hod='predict')
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num_scaled', ...), ('num_binary', ...), ...]"


In [ ]:
print("🔹 Mejor recall promedio (CV):", gs.best_score_)
print("🔹 Mejor combinación de hiperparámetros:", gs.best_params_)

os.makedirs('../models', exist_ok=True)
joblib.dump(gs.best_estimator_, os.path.join('..', 'models', 'best_logreg_balanced.joblib'))
print("Pipeline optimizado y guardado correctamente.")

🔹 Mejor recall promedio (CV): 0.8768697846129806
🔹 Mejor combinación de hiperparámetros: {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}
Pipeline optimizado y guardado correctamente.


**Nota:** el modelo ajustado con class_weight='balanced' y optimizado mediante validación cruzada estratificada (5 folds) alcanzó un recall promedio de 0.876, indicando una alta capacidad para identificar correctamente los casos positivos de diabetes.
La mejor configuración de hiperparámetros fue C=1, penalty='l2' y solver='lbfgs', logrando un equilibrio adecuado entre complejidad y generalización.
Este pipeline se guardó como best_logreg_balanced.joblib.

# Evaluación rápida en el conjunto de test

In [14]:
pipe = joblib.load(os.path.join('..', 'models', 'best_logreg_balanced.joblib'))

y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1] if hasattr(pipe, "predict_proba") else None

print("Reporte de clasificación en TEST:")
print(classification_report(y_test, y_pred))

if y_proba is not None:
    print("ROC AUC (test):", roc_auc_score(y_test, y_proba))

Reporte de clasificación en TEST:
              precision    recall  f1-score   support

           0       0.83      0.90      0.87      8000
           1       0.93      0.88      0.90     12000

    accuracy                           0.89     20000
   macro avg       0.88      0.89      0.88     20000
weighted avg       0.89      0.89      0.89     20000

ROC AUC (test): 0.9339465729166667


**Nota**: el modelo de regresión logística balanceada logró un desempeño sobresaliente en el conjunto de prueba, con un accuracy del 89 % y un AUC de 0.93. Además, mantiene un equilibrio adecuado entre recall (0.88) y precision (0.93), lo que indica que detecta eficazmente la mayoría de los casos con diabetes. 
El modelo tiene una ligeramente mejor capacidad para reconocer casos sin diabetes (recall=0.90) que para detectar casos con diabetes (recall=0.88), lo que indica que aún existe una pequeña proporción de falsos negativos. Sin embargo, la diferencia es mínima (2 puntos porcentuales), lo cual muestra un equilibrio adecuado entre ambas clases.
Estos resultados confirman que el modelo generaliza bien y puede considerarse una herramienta confiable para la predicción del diagnóstico de diabetes en nuevos pacientes.

# Exportar X_train / X_test / y_train / y_test

In [15]:
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

X_train.to_csv(os.path.join(output_dir, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(output_dir, 'X_test.csv'), index=False)
y_train.to_csv(os.path.join(output_dir, 'y_train.csv'), index=False)
y_test.to_csv(os.path.join(output_dir, 'y_test.csv'), index=False)

print("Conjuntos train/test exportados a '../data/processed/'.")

Conjuntos train/test exportados a '../data/processed/'.


**Nota:** los conjuntos X_train, X_test, y_train y y_test fueron exportados a la carpeta data/processed como parte del proceso CRISP–DM. Aunque el modelado posterior se realizó usando los datos cargados en memoria, los archivos exportados permiten garantizar la reproducibilidad del proyecto, facilitar evaluaciones externas y permitir reentrenamientos futuros sin necesidad de repetir el preprocesamiento.